# Reconciling Object & Organisation Types


In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../..")
import os

from heritageconnector.config import config
from heritageconnector.utils.data_transformation import transform_series_str_to_list
from heritageconnector.entity_matching.reconciler import reconciler

from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_pickle("../../GITIGNORE_DATA/results/filtering_people_orgs_result.pkl")
# df_people = df[df['GENDER'].isin(["M", "F"])]
df_orgs = df[df['GENDER'] == "N"]

In [3]:
org_type_col = "OCCUPATION"

In [4]:
df_orgs.loc[:, org_type_col] = df_orgs.loc[:, org_type_col].str.replace("'", "")
df_orgs.loc[:, org_type_col] = transform_series_str_to_list(df_orgs[org_type_col], separator=";")

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [5]:
df_orgs[org_type_col]

0        [manufacturer of mathematical instruments]
6                                   [railway board]
8                                        [supplier]
12                         [training establishment]
14           [manufacturer of electrical equipment]
                            ...                    
18053                                    [hospital]
18061                                            []
18067                      [designer, manufacturer]
18068                                [manufacturer]
18076                                            []
Name: OCCUPATION, Length: 7743, dtype: object

In [7]:
rec = reconciler(df_orgs.head(100), table="ORGANISATION")
resdf = rec.process_column(org_type_col, multiple_vals=True, instanceof_filter="Q43229")

2020-09-29 11:20:11,964 - heritageconnector.entity_matching.reconciler - INFO - Looking up Wikidata qcodes for unique items..


100%|██████████| 92/92 [00:34<00:00,  2.68it/s]


In [16]:
rec._map_df

,count,qid
manufacturer of mathematical instruments,1,[]
railway board,1,[]
supplier,1,[]
training establishment,1,[]
manufacturer of electrical equipment,4,[]
...,...,...
linear measure maker,1,[]
textile manufacturer,1,[]
glass manufacturer,1,[]
cotton milling,1,[]
